In [1]:
import smbus
import math
import time

ModuleNotFoundError: No module named 'smbus'

In [ ]:
# I2C 地址和寄存器地址
HMC5883L_ADDRESS = 0x1E
MPU6050_ADDRESS = 0x68

In [4]:
# HMC5883L寄存器地址
HMC5883L_CONFIG_A = 0x00
HMC5883L_CONFIG_B = 0x01
HMC5883L_MODE = 0x02
HMC5883L_DATA_X_MSB = 0x03
HMC5883L_DATA_Z_MSB = 0x05
HMC5883L_DATA_Y_MSB = 0x07

In [ ]:
# MPU6050寄存器地址
MPU6050_PWR_MGMT_1 = 0x6B
MPU6050_ACCEL_XOUT_H = 0x3B
MPU6050_GYRO_XOUT_H = 0x43

In [ ]:
# 初始化 I2C 总线
bus = smbus.SMBus(1)

In [ ]:
# 初始化 HMC5883L
def initialize_hmc5883l():
    bus.write_byte_data(HMC5883L_ADDRESS, HMC5883L_CONFIG_A, 0x70)  
    bus.write_byte_data(HMC5883L_ADDRESS, HMC5883L_CONFIG_B, 0x20)  
    bus.write_byte_data(HMC5883L_ADDRESS, HMC5883L_MODE, 0x00) 

In [ ]:
# 读取 HMC5883L 数据
def read_magnetometer():
    x = read_word(HMC5883L_DATA_X_MSB)
    z = read_word(HMC5883L_DATA_Z_MSB)
    y = read_word(HMC5883L_DATA_Y_MSB)
    return x, y, z

In [ ]:
# 读取字节数据
def read_word(register):
    high = bus.read_byte_data(HMC5883L_ADDRESS, register)
    low = bus.read_byte_data(HMC5883L_ADDRESS, register + 1)
    val = (high << 8) | low
    if val > 32767:
        val -= 65536
    return val

In [ ]:
# 初始化 MPU6050
def initialize_mpu6050():
    bus.write_byte_data(MPU6050_ADDRESS, MPU6050_PWR_MGMT_1, 0x00)  

In [ ]:
# 读取 MPU6050 数据
def read_accel_gyro():
    ax = read_word_mpu(MPU6050_ACCEL_XOUT_H)
    ay = read_word_mpu(MPU6050_ACCEL_XOUT_H + 2)
    az = read_word_mpu(MPU6050_ACCEL_XOUT_H + 4)
    gx = read_word_mpu(MPU6050_GYRO_XOUT_H)
    gy = read_word_mpu(MPU6050_GYRO_XOUT_H + 2)
    gz = read_word_mpu(MPU6050_GYRO_XOUT_H + 4)
    return ax, ay, az, gx, gy, gz


In [ ]:
# 读取 MPU6050 字节数据
def read_word_mpu(register):
    high = bus.read_byte_data(MPU6050_ADDRESS, register)
    low = bus.read_byte_data(MPU6050_ADDRESS, register + 1)
    val = (high << 8) | low
    if val > 32767:
        val -= 65536
    return val

In [ ]:
offset = -40.0 

In [ ]:
# 计算方向角
def calculate_heading(x, y):
    heading_radians = math.atan2(y, x)
    heading_degrees = math.degrees(heading_radians)
    if heading_degrees < 0:
        heading_degrees += 360
    return heading_degrees

In [ ]:
if __name__ == "__main__":
    initialize_hmc5883l()
    initialize_mpu6050()
    
    try:
        while True:
            x_mag, y_mag, z_mag = read_magnetometer()
            
            ax, ay, az, gx, gy, gz = read_accel_gyro()
            
            heading = calculate_heading(x_mag, y_mag)
            
            print(f"Magnetic Heading: {heading:.2f}°")
            
            print(f"Accelerometer (X, Y, Z): ({ax}, {ay}, {az})")
            print(f"Gyroscope (X, Y, Z): ({gx}, {gy}, {gz})")
            
            time.sleep(1)
    except KeyboardInterrupt:
        print("Program terminated")